In [8]:
from data import _init_corpora

In [9]:
PATH = 'data/small_test_corpora'
TOPIC = 'nyt_covid'
batch_size = 2
time_steps = 20
freq_threshold = 3
epochs = 1

In [10]:
train, valid, test, vocab_dict = _init_corpora(PATH, TOPIC, freq_threshold)

In [11]:
test

array([ 4, 10,  4,  4,  4, 18,  4, 13,  4,  4,  4, 17,  4,  8,  4,  4,  4,
        4,  4,  4,  2,  3,  4,  5,  4,  4, 13,  4,  4,  4,  4,  4,  4,  4,
       18,  4, 17,  4,  2,  3, 17,  4,  4,  4,  4,  4,  4,  4,  4,  1, 17,
        4,  4,  4,  4,  4,  4, 14,  4,  4,  4,  1,  7,  4,  4,  4,  4,  4,
        2,  3,  4,  4,  4, 13, 17,  4,  1,  4,  8,  4,  4, 18,  4,  4,  4,
        4,  2,  3,  4,  4,  4,  4,  4,  4,  4,  4,  4, 18,  4,  1,  4,  4,
        4,  4,  4,  4,  4, 13,  4,  4,  4,  4,  7,  4,  4,  1,  4,  4,  4,
        4,  4,  4,  4,  4,  2,  3])

In [28]:
def minibatch(data, batch_size, seq_length):
    data = torch.tensor(data, dtype = torch.int64)
    num_batches = data.size(0)//batch_size
    data = data[:num_batches*batch_size]
    data=data.view(batch_size,-1)
    dataset = []
    for i in range(0,data.size(1)-1,seq_length):
        seqlen=int(np.min([seq_length,data.size(1)-1-i]))
        if seqlen<data.size(1)-1-i:
            x=data[:,i:i+seqlen].transpose(1, 0)
            y=data[:,i+1:i+seqlen+1].transpose(1, 0)
            dataset.append((x, y))
    return dataset

In [34]:
d = minibatch(test, 1, 6)
d


[tensor([[ 4],
         [10],
         [ 4],
         [ 4],
         [ 4],
         [18]]),
 tensor([[ 4],
         [13],
         [ 4],
         [ 4],
         [ 4],
         [17]]),
 tensor([[4],
         [8],
         [4],
         [4],
         [4],
         [4]]),
 tensor([[4],
         [4],
         [2],
         [3],
         [4],
         [5]]),
 tensor([[ 4],
         [ 4],
         [13],
         [ 4],
         [ 4],
         [ 4]]),
 tensor([[ 4],
         [ 4],
         [ 4],
         [ 4],
         [18],
         [ 4]]),
 tensor([[17],
         [ 4],
         [ 2],
         [ 3],
         [17],
         [ 4]]),
 tensor([[4],
         [4],
         [4],
         [4],
         [4],
         [4]]),
 tensor([[ 4],
         [ 1],
         [17],
         [ 4],
         [ 4],
         [ 4]]),
 tensor([[ 4],
         [ 4],
         [ 4],
         [14],
         [ 4],
         [ 4]]),
 tensor([[4],
         [1],
         [7],
         [4],
         [4],
         [4]]),
 tensor([[4]